# Compaction 전략 실습 — Binpack, Sort, Z-order

이 노트북에서는 Iceberg의 **Compaction(컴팩션)** 개념과 3가지 전략(Binpack, Sort, Z-order)을 실습합니다.

## Compaction이란?

Compaction(컴팩션)은 **여러 개의 작은 파일을 더 적은 수의 큰 파일로 재작성**하는 프로세스입니다.

### 왜 필요한가?

데이터를 읽을 때 두 가지 비용이 발생합니다:
- **고정 비용**: 쿼리에 필요한 데이터를 실제로 읽는 비용 (피할 수 없음)
- **가변 비용**: 파일을 열고, 스캔하고, 닫는 파일 작업 비용 (줄일 수 있음!)

파일이 많을수록 가변 비용이 증가합니다. 특히 스트리밍 환경에서는 각각 몇 개의 레코드만 담은 수많은 작은 파일이 생성되어 **Small File Problem**이 심각해집니다.

### 3가지 컴팩션 전략

| 전략 | 동작 | 장점 | 단점 |
|------|------|------|------|
| **Binpack** | 파일만 결합 (정렬 없음) | 가장 빠름 | 데이터 순서 변경 없음 |
| **Sort** | 지정 필드로 순차 정렬 후 결합 | 단일 필드 프루닝 가능 | Binpack보다 느림 |
| **Z-order** | 여러 필드를 동등 가중치로 정렬 | 복합 필터 프루닝 가능 | Binpack보다 느림 |

## 환경 설정

In [ ]:
import sys
sys.path.append('..')

import time
from utils.spark_setup import create_spark_session
from utils.data_generator import generate_orders, to_spark_df
from utils.file_explorer import show_tree, snapshot_tree, diff_tree, count_files, total_size

In [ ]:
spark = create_spark_session()

# 테이블 설정
TABLE_NAME = "demo.lab.compact_orders"
TABLE_PATH = "/home/jovyan/data/warehouse/lab/compact_orders"

SORT_TABLE = "demo.lab.compact_sort_orders"
SORT_TABLE_PATH = "/home/jovyan/data/warehouse/lab/compact_sort_orders"

ZORDER_TABLE = "demo.lab.compact_zorder_orders"
ZORDER_TABLE_PATH = "/home/jovyan/data/warehouse/lab/compact_zorder_orders"

---
## 실험 1: Small File Problem 시연

10건씩 10번 반복 INSERT하여 작은 파일이 폭증하는 현상을 관찰합니다.

In [ ]:
# Binpack 실험용 테이블 생성
spark.sql(f"DROP TABLE IF EXISTS {TABLE_NAME}")

spark.sql(f"""
CREATE TABLE {TABLE_NAME} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

print(f"테이블 생성 완료: {TABLE_NAME}")

In [ ]:
# 10건씩 10번 반복 INSERT → 작은 파일 폭증
for i in range(10):
    orders = generate_orders(num_records=10, seed=i, id_offset=i*10+1)
    df = to_spark_df(spark, orders)
    df.writeTo(TABLE_NAME).append()
    print(f"배치 {i+1}/10 완료 (10건 삽입)")

print(f"\n총 레코드 수: {spark.sql(f'SELECT COUNT(*) FROM {TABLE_NAME}').collect()[0][0]}")

In [ ]:
# Small File Problem 확인
print("파일 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

parquet_count = count_files(TABLE_PATH)
total = total_size(TABLE_PATH)
print(f"\nParquet 파일 수: {parquet_count}")
print(f"총 크기: {total:,} bytes")

In [ ]:
# 쿼리 성능 측정 (컴팩션 전)
start = time.time()
spark.sql(f"SELECT * FROM {TABLE_NAME}").collect()
elapsed_before = time.time() - start
print(f"전체 테이블 SELECT 소요 시간 (컴팩션 전): {elapsed_before:.3f}초")
print(f"스캔 파일 수: {parquet_count}")

### 관찰 포인트 — Small File Problem

- 10번의 INSERT로 각 파티션마다 최대 10개의 작은 파일이 생성되었습니다
- 100건의 작은 데이터에 비해 파일 수가 과도하게 많습니다
- 이는 스트리밍 수집이나 빈번한 소량 INSERT에서 흔히 발생하는 문제입니다

---
## 실험 2: Binpack 전략

Binpack은 가장 빠른 컴팩션 전략으로, **데이터 순서를 변경하지 않고 파일만 결합**합니다.

In [ ]:
before = snapshot_tree(TABLE_PATH)

# Binpack 컴팩션 실행
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{TABLE_NAME}',
    strategy => 'binpack',
    options => map('target-file-size-bytes', '134217728')
)
""").show(truncate=False)

after = snapshot_tree(TABLE_PATH)

print("=" * 60)
print("Binpack 컴팩션 전후 변경 사항:")
print("=" * 60)
diff_tree(before, after)

In [ ]:
# 컴팩션 후 파일 구조
print("Binpack 후 파일 구조:")
print("=" * 60)
show_tree(TABLE_PATH)

new_parquet_count = count_files(TABLE_PATH)
new_total = total_size(TABLE_PATH)
print(f"\nParquet 파일 수: {parquet_count} → {new_parquet_count}")
print(f"총 크기: {total:,} → {new_total:,} bytes")

In [ ]:
# 쿼리 성능 재측정
start = time.time()
spark.sql(f"SELECT * FROM {TABLE_NAME}").collect()
elapsed_after = time.time() - start
print(f"전체 테이블 SELECT 소요 시간 (컴팩션 전): {elapsed_before:.3f}초")
print(f"전체 테이블 SELECT 소요 시간 (컴팩션 후): {elapsed_after:.3f}초")
print(f"스캔 파일 수: {parquet_count} → {new_parquet_count}")

### 관찰 포인트 — Binpack

- **파일 수가 크게 감소**했습니다 (파티션당 1개로 통합)
- 데이터 순서는 변경되지 않았습니다 — 단순히 파일만 합쳤습니다
- Binpack은 **가장 빠른 전략**으로, 스트리밍 SLA를 충족해야 할 때 적합합니다
- 파일 수 감소로 쿼리의 파일 작업(open/scan/close) 비용이 줄어듭니다

---
## 실험 3: Sort 전략

Sort 전략은 데이터를 **지정 필드로 정렬한 후 결합**합니다. 정렬하면 유사한 값이 적은 파일에 집중되어, manifest의 min/max 통계를 통한 **파일 프루닝**이 가능해집니다.

In [ ]:
# Sort 실험용 테이블 생성 + 데이터 삽입
spark.sql(f"DROP TABLE IF EXISTS {SORT_TABLE}")

spark.sql(f"""
CREATE TABLE {SORT_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 10건씩 10번 반복 INSERT (작은 파일 생성)
for i in range(10):
    orders = generate_orders(num_records=10, seed=100+i, id_offset=i*10+1)
    df = to_spark_df(spark, orders)
    df.writeTo(SORT_TABLE).append()

print(f"테이블 생성 + 100건 삽입 완료: {SORT_TABLE}")
print(f"Parquet 파일 수: {count_files(SORT_TABLE_PATH)}")

In [ ]:
before = snapshot_tree(SORT_TABLE_PATH)

# Sort 컴팩션 실행 (product_name 기준 정렬)
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{SORT_TABLE}',
    strategy => 'sort',
    sort_order => 'product_name ASC NULLS LAST'
)
""").show(truncate=False)

after = snapshot_tree(SORT_TABLE_PATH)

print("=" * 60)
print("Sort 컴팩션 전후 변경 사항:")
print("=" * 60)
diff_tree(before, after)

In [ ]:
# Sort 후 파일 구조 확인
print("Sort 후 파일 구조:")
print("=" * 60)
show_tree(SORT_TABLE_PATH)

print(f"\nParquet 파일 수: {count_files(SORT_TABLE_PATH)}")

In [ ]:
# Manifest의 column stats 확인 — 정렬 후 min/max 범위가 좁아졌는지 확인
print("Manifest 파일 정보:")
spark.sql(f"SELECT * FROM {SORT_TABLE}.manifests").show(truncate=False)

In [ ]:
# 특정 product_name으로 필터링 — 파일 프루닝 효과 확인
print("EXPLAIN: product_name = 'iPhone 14' 필터 쿼리")
spark.sql(f"EXPLAIN EXTENDED SELECT * FROM {SORT_TABLE} WHERE product_name = 'iPhone 14'").show(truncate=False)

### 관찰 포인트 — Sort

- 데이터가 `product_name` 기준으로 **정렬되어 재작성**되었습니다
- 정렬 후 각 파일의 min/max 범위가 좁아져, 특정 값 필터링 시 **불필요한 파일을 건너뛸 수 있습니다** (파일 프루닝)
- EXPLAIN 결과에서 스캔 대상 파일이 줄어든 것을 확인할 수 있습니다
- Sort는 **자주 필터링하는 단일 필드**가 있을 때 가장 효과적입니다

---
## 실험 4: Z-order 전략

Z-order는 **여러 필드를 동등한 가중치로 다차원 정렬**합니다.

### Z-order 개념

일반 정렬은 1차원적입니다 — A 기준으로 정렬하면 B는 순서가 보장되지 않습니다.

Z-order는 **사분면 기반 탐색**을 사용합니다:
1. X, Y 두 축의 값 범위를 4개 사분면으로 나눕니다
2. 검색값의 사분면을 확인하면, 나머지 3개 사분면(75%!)을 즉시 제외할 수 있습니다
3. 사분면을 재귀적으로 세분화하여 검색 범위를 좁힙니다

이 방식은 `WHERE product_name = 'X' AND customer_id = 123`처럼 **복합 필터 쿼리**에서 큰 효과를 발휘합니다.

In [ ]:
# Z-order 실험용 테이블 생성 + 데이터 삽입
spark.sql(f"DROP TABLE IF EXISTS {ZORDER_TABLE}")

spark.sql(f"""
CREATE TABLE {ZORDER_TABLE} (
    order_id BIGINT,
    customer_id BIGINT,
    product_name STRING,
    order_date DATE,
    amount DECIMAL(10,2),
    status STRING
) USING ICEBERG
PARTITIONED BY (months(order_date))
""")

# 10건씩 10번 반복 INSERT
for i in range(10):
    orders = generate_orders(num_records=10, seed=200+i, id_offset=i*10+1)
    df = to_spark_df(spark, orders)
    df.writeTo(ZORDER_TABLE).append()

print(f"테이블 생성 + 100건 삽입 완료: {ZORDER_TABLE}")
print(f"Parquet 파일 수: {count_files(ZORDER_TABLE_PATH)}")

In [ ]:
before = snapshot_tree(ZORDER_TABLE_PATH)

# Z-order 컴팩션 실행 (product_name + customer_id 기준)
spark.sql(f"""
CALL demo.system.rewrite_data_files(
    table => '{ZORDER_TABLE}',
    strategy => 'sort',
    sort_order => 'zorder(product_name, customer_id)'
)
""").show(truncate=False)

after = snapshot_tree(ZORDER_TABLE_PATH)

print("=" * 60)
print("Z-order 컴팩션 전후 변경 사항:")
print("=" * 60)
diff_tree(before, after)

In [ ]:
# Z-order 후 파일 구조 확인
print("Z-order 후 파일 구조:")
print("=" * 60)
show_tree(ZORDER_TABLE_PATH)

print(f"\nParquet 파일 수: {count_files(ZORDER_TABLE_PATH)}")

In [ ]:
# 복합 필터 쿼리의 프루닝 효과 확인
print("EXPLAIN: product_name + customer_id 복합 필터 쿼리")
spark.sql(f"""
EXPLAIN EXTENDED 
SELECT * FROM {ZORDER_TABLE} 
WHERE product_name = 'iPhone 14' AND customer_id = 500
""").show(truncate=False)

### 관찰 포인트 — Z-order

- Z-order는 `product_name`과 `customer_id` **두 필드를 동시에 고려**하여 정렬했습니다
- 복합 필터 쿼리(`WHERE A = x AND B = y`)에서 **프루닝 효과가 극대화**됩니다
- 단일 필드 필터에서는 Sort보다 효과가 떨어질 수 있지만, 여러 필드 조합 필터에서 강점을 발휘합니다

---
## 타겟 컴팩션과 고급 옵션

### 타겟 컴팩션 (Where 필터)

전체 테이블이 아닌 **특정 파티션이나 조건에 맞는 데이터만** 컴팩션할 수 있습니다:

```sql
CALL catalog.system.rewrite_data_files(
    table => 'my_table',
    strategy => 'binpack',
    where => 'order_date >= "2024-01-01"'
)
```

스트리밍 환경에서 최근 수집된 파티션만 주기적으로 컴팩션하면 효율적입니다.

### File Groups & Partial Progress

- **File Groups**: 컴팩션 대상 파일을 그룹으로 나눠 병렬 처리합니다
  - `max-file-group-size-bytes`: 그룹 최대 크기
  - `max-concurrent-file-group-rewrites`: 동시 처리 그룹 수

- **Partial Progress**: 파일 그룹 완료 시마다 새 스냅샷을 생성합니다
  - `partial-progress-enabled`: 부분 진행 활성화
  - `partial-progress-max-commits`: 최대 커밋 수
  - 장점: 다른 쿼리가 컴팩션 진행 중에도 이미 완료된 부분의 이점을 얻을 수 있음
  - 주의: 더 많은 스냅샷 = 더 많은 메타데이터 파일

---
## 전략 비교 요약

| 전략 | 속도 | 읽기 개선 | 적합 사례 |
|------|------|-----------|----------|
| **Binpack** | 가장 빠름 | 파일 수만 감소 | 스트리밍 SLA, 빠른 컴팩션 필요 시 |
| **Sort** | 중간 | 단일 필드 프루닝 | 자주 필터링하는 필드가 명확할 때 |
| **Z-order** | 중간 | 다중 필드 프루닝 | 복합 필터 쿼리가 빈번할 때 |

### 전략 선택 가이드

1. **"일단 파일 수를 줄이고 싶다"** → Binpack
2. **"특정 컬럼으로 자주 필터링한다"** → Sort
3. **"여러 컬럼 조합으로 필터링한다"** → Z-order
4. **"스트리밍 SLA가 빡빡하다"** → Binpack (속도 우선)

> 컴팩션은 일회성이 아닙니다. 데이터가 계속 쌓이면 주기적으로 실행해야 합니다.
> Airflow, Dagster 등 오케스트레이션 도구로 자동화하거나, 관리형 카탈로그(Tabular 등)의 자동 유지보수 기능을 활용하세요.

In [ ]:
spark.stop()
print("Spark 세션 종료")